<a href="https://colab.research.google.com/github/ykato27/ai-demo/blob/main/6DRepNet_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

論文  
https://arxiv.org/abs/2202.12555<br>
<br>  
GitHub  
https://github.com/thohemp/6drepnet<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/6DRepNet_demo.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 環境セットアップ

## GPU確認

In [1]:
!nvidia-smi

Sun Apr 23 00:33:57 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## GitHubからコード取得

In [2]:
%cd /content

!git clone https://github.com/thohemp/6DRepNet

/content
Cloning into '6DRepNet'...
remote: Enumerating objects: 323, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 323 (delta 112), reused 109 (delta 93), pack-reused 182
Receiving objects: 100% (323/323), 97.68 KiB | 13.95 MiB/s, done.
Resolving deltas: 100% (182/182), done.


## ライブラリのインストール

In [3]:
%cd /content/6DRepNet

!pip install --upgrade gdown
!pip install git+https://github.com/elliottzheng/face-detection.git@master

/content/6DRepNet
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.6.6
    Uninstalling gdown-4.6.6:
      Successfully uninstalled gdown-4.6.6
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/elliottzheng/face-detection.git (to revision master) to /tmp/pip-req-build-ecpg5vys
  Running command git clone --filter=blob:none --quiet https://github.com/elliottzheng/face-detection.git /tmp/pip-req-build-ecpg5vys
  Resolved https://github.com/elliottzheng/face-detection.git to commit bf24ac378eb7da0f302f65fed43e5253d1cbe09b
  Preparing metadata (setup.py) ... done
  Created wheel for face-detection: filename=face_detection-1.0.5-py3-none-any.whl size=1638328 sha256=9a1e4c9a7ecc0404811d5b7430d06b1a17b73a36e1085a89e61306efa7f6d17e
  Stored in directory: /tmp/pip-ephem-wheel-cache-eznvse8s/wheels/4

## ライブラリのインポート

In [4]:
%cd /content/6DRepNet/sixdrepnet

/content/6DRepNet/sixdrepnet


In [5]:
%cd /content/6DRepNet/sixdrepnet
from model import SixDRepNet
import math
import re
from matplotlib import pyplot as plt
import sys
import os
import argparse

import numpy as np
import cv2
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
from numpy.lib.function_base import _quantile_unchecked

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
import torch.backends.cudnn as cudnn
import torchvision
import torch.nn.functional as F
import utils
import matplotlib
from PIL import Image
import time
from face_detection import RetinaFace

import glob
from google.colab import files
from tqdm import tqdm

## 学習済みモデルのダウンロード

In [6]:
%cd /content/6DRepNet
!mkdir pretrained

#https://drive.google.com/file/d/1vPNtVu_jg2oK-RiIWakxYyfLPA9rU4R4/view?usp=sharing
pretrained_ckpt = 'pretrained/6DRepNet_300W_LP_AFLW2000.pth'
if not os.path.exists(pretrained_ckpt):
  !gdown --id 1vPNtVu_jg2oK-RiIWakxYyfLPA9rU4R4 \
          -O {pretrained_ckpt}

snapshot_path = os.path.join("/content/6DRepNet/", "pretrained/6DRepNet_300W_LP_AFLW2000.pth")

/content/6DRepNet
/usr/local/lib/python3.9/dist-packages/gdown/cli.py:126: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (uriginal): https://drive.google.com/uc?id=1vPNtVu_jg2oK-RiIWakxYyfLPA9rU4R4
From (redirected): https://drive.google.com/uc?id=1vPNtVu_jg2oK-RiIWakxYyfLPA9rU4R4&confirm=t&uuid=88667060-6086-47c5-adbf-345ba9646edd
To: /content/6DRepNet/pretrained/6DRepNet_300W_LP_AFLW2000.pth
100% 157M/157M [00:07<00:00, 21.8MB/s]


# テスト動画のセットアップ

## 動画のアップロード
使用動画<br>
https://www.pexels.com/ja-jp/video/3201691/

In [7]:
%cd /content/6DRepNet
!rm -rf upload
!mkdir -p upload/frames
%cd upload

uploaded = files.upload()
uploaded = list(uploaded.keys())
file_name = uploaded[0]

upload_path = os.path.join("/content/6DRepNet/upload", file_name)
print("upload file here:", upload_path)

/content/6DRepNet
/content/6DRepNet/upload


Saving pexels-cottonbro-studio-3201691-1920x1080-25fps.mp4 to pexels-cottonbro-studio-3201691-1920x1080-25fps.mp4
upload file here: /content/6DRepNet/upload/pexels-cottonbro-studio-3201691-1920x1080-25fps.mp4


## 動画をフレーム画像に分割

In [8]:
%cd /content/6DRepNet/upload

!ffmpeg -i {upload_path} frames/%06d.png

frames = glob.glob("/content/6DRepNet/upload/frames/*.png")

/content/6DRepNet/upload
ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidst

# Head Pose Estimation

In [9]:
%cd /content/6DRepNet

!rm -rf output
!mkdir -p output/frames

cudnn.enabled = True
gpu = 0

print("Start model setup...")
# Modelのビルド
model = SixDRepNet(
    backbone_name='RepVGG-B1g2',
    backbone_file='',
    deploy=True,
    pretrained=False)

detector = RetinaFace(gpu_id=gpu)

# Modelのロード
saved_state_dict = torch.load(os.path.join(snapshot_path), map_location='cpu')

if 'model_state_dict' in saved_state_dict:
  model.load_state_dict(saved_state_dict['model_state_dict'])
else:
  model.load_state_dict(saved_state_dict)    
  model.cuda(gpu)

# Test the Model
model.eval()  # Change model to 'eval' mode (BN uses moving mean/var).

print("Complete model setup.")

print("loading ", len(frames), " frames...")

process_start = time.time()
with torch.no_grad():
  for i in tqdm( range(len(frames)) ):
    img_path = frames[i]
    frame = np.array(Image.open(img_path))

    faces = detector(frame)

    for box, landmarks, score in faces:
      # Print the location of each face in this image
      if score < .95:
          continue
      x_min = int(box[0])
      y_min = int(box[1])
      x_max = int(box[2])
      y_max = int(box[3])         
      bbox_width = abs(x_max - x_min)
      bbox_height = abs(y_max - y_min)

      x_min = max(0,x_min-int(0.2*bbox_height))
      y_min = max(0,y_min-int(0.2*bbox_width))
      x_max = x_max+int(0.2*bbox_height)
      y_max = y_max+int(0.2*bbox_width)

      img = frame[y_min:y_max,x_min:x_max]

      img = cv2.resize(img, (244, 244))/255.0
      img = img.transpose(2, 0, 1)
      img = torch.from_numpy(img).type(torch.FloatTensor)
      img = torch.Tensor(img).cuda(gpu)
      img=img.unsqueeze(0)

      start = time.time()
      R_pred = model(img)
      end = time.time()
      #print('Head pose estimation: %2f ms'% ((end - start)*1000.))

      euler = utils.compute_euler_angles_from_rotation_matrices(R_pred)*180/np.pi
      p_pred_deg = euler[:, 0].cpu()
      y_pred_deg = euler[:, 1].cpu()
      r_pred_deg = euler[:, 2].cpu()

      utils.plot_pose_cube(frame,  y_pred_deg, p_pred_deg, r_pred_deg, x_min + int(.5*(x_max-x_min)), y_min + int(.5*(y_max-y_min)), size = bbox_width)

    # 1フレーム完了毎に表示する場合はコメントアウト解除
    #cv2_imshow(frame)
    cv2.imwrite( os.path.join("/content/6DRepNet/output/frames", os.path.basename(img_path)), frame)

process_end = time.time()
print('Complete All Head pose estimation: %2f s'% (process_end - process_start))
print('Average %2f ms/ %06d frames'% (((process_end - process_start)*1000.)/len(frames), len(frames)))

/content/6DRepNet
Start model setup...
Complete model setup.
loading  239  frames...


100%|██████████| 239/239 [01:20<00:00,  2.96it/s]

Complete All Head pose estimation: 80.875932 s
Average 338.393023 ms/ 000239 frames


## フレーム画像を動画に変換

In [10]:
!ffmpeg -i "/content/6DRepNet/output/frames/%06d.png" -c:v libx264 -vf "format=yuv420p" "/content/6DRepNet/output/result.mp4"

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

## Head Pose Estimationの結果を表示

In [11]:
from moviepy.editor import *
from moviepy.video.fx.resize import resize
clip = VideoFileClip("/content/6DRepNet/output/result.mp4")
clip = resize(clip, height=420)
clip.ipython_display()

Moviepy - Building video __temp__.mp4.
Moviepy - Writing video __temp__.mp4



Moviepy - Done !
Moviepy - video ready __temp__.mp4
